In [10]:
import os
import pdfplumber
import pandas as pd
from gpt_bot_driver import gpt_bot_driver
import time
pd.set_option('display.max_colwidth', None)

data_dir = '../data'
fnames = os.listdir(data_dir)

# gpt 챗봇 실행
bot = gpt_bot_driver("haesung.kim")

for fname in fnames:
    if fname.endswith('.pdf'):        
        print('processing', fname)
        # 텍스트 파일을 작성할 경로와 파일명 지정
        input_path = os.path.join(data_dir, fname)
        output_path = os.path.join(data_dir, fname).rstrip('.pdf') +'.txt'
    
        # 텍스트 파일을 쓰기 모드로 열기
        with open(output_path, "w", encoding='utf-8') as file:
            pdf = pdfplumber.open(input_path)            
            for page_index, page in enumerate(pdf.pages):
                
                page_text = page.extract_text()
                table = page_text.maketrans('\n',' ')
                page_text_processed = page_text.translate(table)
                
                tables = page.extract_tables(
                    table_settings = {
                        "vertical_strategy": "lines", 
                        "horizontal_strategy": "lines",
                    }
                )
                file.write(f'/----------------------------------------------------- Page {page_index+1} Start -----------------------------------------------------\\\n')
                file.write(f'-----------------------------------------------------  Raw Text  -----------------------------------------------------\n')
                file.write(page_text+'\n')
                file.write(f'-----------------------------------------------------English Text-----------------------------------------------------\n')
                bot.delete_conversation_history() # 대화 이력 삭제
                time.sleep(3)
                question = ' 앞의 글을 영어로 번역해줘, 영어로만 답변해줘'
                answer1 = bot.get_gpt_answer(page_text_processed + ' ' + question) # 질의 진행
                file.write(answer1+'\n')
                file.write('\n')
                file.write(f'----------------------------------------------------- Korean Text-----------------------------------------------------\n')
                bot.delete_conversation_history() # 대화 이력 삭제
                time.sleep(3)
                question = ' 앞의 글을 한국어로 번역해줘, 한국어로만 답변해줘'
                answer2 = bot.get_gpt_answer(page_text_processed + ' ' + question) # 질의 진행
                file.write(answer2+'\n')
                file.write('\n')
                file.write(f'--------------------------------------------------- Summarization ---------------------------------------------------\n')
                bot.delete_conversation_history() # 대화 이력 삭제
                question = ' 앞의 글을 개조식으로 요약해줘'
                time.sleep(3)
                answer3 = bot.get_gpt_answer(answer2 + ' ' + question) # 질의 진행
                file.write(answer3+'\n')
                file.write('\n')
                file.write(f'------------------------------------------------- Extracted Keyword -------------------------------------------------\n')
                bot.delete_conversation_history() # 대화 이력 삭제
                question = ' 앞의 글에서 중요한 핵심 키워드들을 [] 안에 뽑아줘'
                time.sleep(3)
                answer4 = bot.get_gpt_answer(answer1 + ' ' + question) # 질의 진행
                file.write(answer4+'\n')
                file.write('\n')
                for table_index, table in enumerate(tables):
                    file.write(f'-----------------------------------------------------  Raw Table  -----------------------------------------------------\n')
                    file.write(f'Table {page_index+1}-{table_index+1}\n')
                    # Convert list of lists to dataframe
                    df = pd.DataFrame(table)
                    file.write(df.to_string(index=False))
                    file.write('\n')
                    # print(f'-----------------------------------------------------English Table-----------------------------------------------------')
                    # print()
                    # print(f'----------------------------------------------------- Korean Table-----------------------------------------------------')
                    # print()
                file.write(f'\\------------------------------------------------------ Page {page_index+1} End ------------------------------------------------------/\n')
                file.write('\n')
        

processing Workstation monitor_24003PPT.pdf
processing 四川省肿瘤医院医用显示器招标文件.pdf


In [33]:
import os
import pdfplumber
import pandas as pd
from gpt_bot_driver import gpt_bot_driver
import time
from docx import Document
from docx.enum.text import WD_ALIGN_PARAGRAPH
from docx.enum.style import WD_STYLE_TYPE
pd.set_option('display.max_colwidth', None)

data_dir = '../data'
fnames = os.listdir(data_dir)

# gpt 챗봇 실행
bot = gpt_bot_driver("haesung.kim")

for fname in fnames:
    if fname.endswith('.pdf'):        
        print('processing', fname)
        # 텍스트 파일을 작성할 경로와 파일명 지정
        input_path = os.path.join(data_dir, fname)
        output_path = os.path.join(data_dir, fname).rstrip('.pdf') +'.docx'

        doc = Document()

        pdf = pdfplumber.open(input_path)            
        for page_index, page in enumerate(pdf.pages):
            # 텍스트 추출
            doc.add_heading(f'Page {page_index+1}', level=0)
            doc.add_heading(f'Raw Text', level=1)
            doc.add_paragraph(page_text)
            doc.add_page_break()
            page_text = page.extract_text()
            
            
            # page \n 제거
            table = page_text.maketrans('\n',' ')
            page_text_processed = page_text.translate(table)
            
            tables = page.extract_tables(
                table_settings = {
                    "vertical_strategy": "lines", 
                    "horizontal_strategy": "lines",
                }
            )
            
            # 영어 번역
            doc.add_heading(f'Page {page_index+1}', level=0)
            doc.add_heading(f'English Text', level=1)
            bot.delete_conversation_history() # 대화 이력 삭제
            time.sleep(3)
            question = ' Please translate the previous text into English and respond only in English.'
            answer1 = bot.get_gpt_answer(page_text_processed + ' ' + question) # 질의 진행
            doc.add_paragraph(answer1)
            doc.add_page_break()
            # 한국어 번역
            doc.add_heading(f'Page {page_index+1}', level=0)
            doc.add_heading(f'Korean Text', level=1)
            bot.delete_conversation_history() # 대화 이력 삭제
            time.sleep(3)
            question = ' 앞의 글을 한국어로 번역해줘, 한국어로만 답변해줘'
            answer2 = bot.get_gpt_answer(page_text_processed + ' ' + question) # 질의 진행
            doc.add_paragraph(answer2)
            doc.add_page_break()
            
            # answer1 \n 제거
            table = answer1.maketrans('\n',' ')
            answer1_processed = answer1.translate(table)
            
            tables = page.extract_tables(
                table_settings = {
                    "vertical_strategy": "lines", 
                    "horizontal_strategy": "lines",
                }
            )

            # 영어 요약
            doc.add_heading(f'Page {page_index+1}', level=0)
            doc.add_heading(f'Summarization', level=1)
            bot.delete_conversation_history() # 대화 이력 삭제
            time.sleep(3)
            question = ' Please summarize the previous text and respond only in English.'
            answer3 = bot.get_gpt_answer(answer1_processed + ' ' + question) # 질의 진행
            doc.add_paragraph(answer3)
            doc.add_page_break()
            # 영어 키워드 추출
            doc.add_heading(f'Page {page_index+1}', level=0)
            doc.add_heading(f'Extracted Keyword', level=1)
            bot.delete_conversation_history() # 대화 이력 삭제
            time.sleep(3)
            question = ' Please extract important keywords from the previous text and respond only in English.'
            answer4 = bot.get_gpt_answer(answer1_processed + ' ' + question) # 질의 진행
            doc.add_paragraph(answer4)
            doc.add_page_break()
            
            if len(tables):
                doc.add_heading(f'Page {page_index+1}', level=0)
                doc.add_heading(f'Raw Table', level=1)    
                for table_index, table in enumerate(tables):
                    doc.add_heading(f'Table {page_index+1}-{table_index+1}', level=2)
                    
                    # Convert list of lists to dataframe
                    df = pd.DataFrame(table)
                    # add a table to the end and create a reference variable
                    # extra row is so we can add the header row
                    t = doc.add_table(df.shape[0]+1, df.shape[1])

                    for j in range(df.shape[-1]):
                        t.cell(0,j).text = str(df.columns[j])

                    for i in range(df.shape[0]):
                        for j in range(df.shape[-1]):
                            t.cell(i+1,j).text = str(df.values[i,j])
                    # print(f'-----------------------------------------------------English Table-----------------------------------------------------')
                    # print()
                    # print(f'----------------------------------------------------- Korean Table-----------------------------------------------------')
                    # print()
                doc.add_page_break()
        doc.save(output_path)
    

processing Workstation monitor_24003PPT.pdf
processing 四川省肿瘤医院医用显示器招标文件.pdf


KeyboardInterrupt: 

In [16]:
text = '''1
PLIEGO DE PRESCRIPCIONES TÉCNICAS
EXPEDIENTE ANTICIPADO DE GASTO: AB-SER2-24-003
SUMINISTRO DE ESTACIONES DE TRABAJO PARA DIAGNÓSTICO MEDIANTE IMAGEN DIGITAL DE ANATOMÍA
PATOLÓGICA CON DESTINO A HOSPITALES DEL SERVICIO GALLEGO DE SALUD, SUSCEPTIBLE DE COFINANCIACIÓN
POR LA UNIÓN EUROPEA EN UN 60% A TRAVÉS DEL PROGRAMA DE GALICIA FEDER 2021-2027, OBJETIVO POLÍTICO
OP 1, OBJETIVO ESPECÍFICO OE. 1.2 Y ACTUACIÓN 1.2.02
Los productos presentados a este procedimiento serán totalmente nuevos y no podrán tener ningún componente
usado o reutilizado. Deberán cumplir la normativa española y comunitaria vigente que sea de aplicación, siendo
asimismo de total responsabilidad del proveedor la obtención de los certificados de homologación o declaración de
conformidad CE correspondientes.
Se ajustarán a la directiva 2001/95/CE relativa a la seguridad general de los productos (R.D. 1801/2003, de 26 de
diciembre, sobre seguridad general de los productos B.O.E. de 10 de enero de 2004).
Cuando los bienes hayan sido fabricados de conformidad a normas (ISO, EN y nacionales o equivalentes, así como
Reglamentos, Directivas y normas de facto), estas se detallarán en la oferta. Asimismo, se indicará si estos bienes
están certificados con estas normas por un organismo de evaluación de conformidad acreditado por ENAC u
organismo de acreditación equivalente, o en su defecto, por empresa independiente.
Deberán cumplir la legislación vigente relativa a productos sanitarios (Real Decreto 1591/2009 de 16 de Octubre) que
les sea de aplicación llevando el marcado CE (en base a la directiva 2007/47/CE) en donde corresponda, acompañado
del número de identificación del organismo notificado (con excepción en los productos de Clase I). Se incluirán
documentos acreditativos del cumplimiento de los requisitos establecidos en la legislación vigente.
En caso de que el equipo genere algún tipo de información clínica (imagen médica, señales fisiológicas u otros datos
clínicos) susceptible de ser incorporada a los sistemas de información del Servizo Galego de Saúde, deberán
cumplirse los requisitos de conectividad y criterios técnicos que sean de aplicación según lo recogido en el
documento de especificaciones técnicas accesible desde la página web (www.sergas.es) del Servizo Galego de Saúde,
en su versión vigente a fecha de publicación en DOG de este pliego.
Deberán describir todos los artículos o accesorios incluidos en su oferta, siguiendo la relación de características que
figuran en este pliego. Los datos aportados deberán corresponder a equipos o accesorios incluidos en la oferta. Se
presentará como información adicional la hoja o catálogo de productos identificando el equipo ofertado para su
máxima descripción técnica.
Todos los datos relativos a componentes opcionales, posibles ampliaciones, y otros elementos no incluidos deberán
identificarse claramente como elementos no incluidos en la oferta y en caso contrario, el órgano de contratación
podrá exigir su entrega como parte integrante de la oferta.
Página 1/6'''

In [17]:
from langchain_community.llms import CTransformers
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

 # mistral
llm_mistral = CTransformers(
    model = "../model/mistral-7b-instruct-v0.1.Q4_K_M.gguf",
    model_type = "mistral",
    hf=True,
    max_new_tokens=10000,
    config={'context_length': 2048}
)

template = """Translate this text to English:

{query}

Just return translated english text.

Translated Text: """

prompt = PromptTemplate(input_variables=["query"], template=template)
to_eng_chain = LLMChain(llm=llm_mistral, prompt=prompt)

to_eng_chain.invoke({"query": text})["text"]

"\n\nTechnical Specifications Anticipated Budget Document No.: AB-SER2-24-003\nSupply of Stations for Diagnostic Imaging with Digital Anatomy in Galicia, Spain, Subject to Financing by the European Union through the EU 60% of the GALICIA FEDER 2021-2027 Programme Objective (OP 1, Specific Objective OE 1.2 and Action 1.2.02)\nThe products presented to this procedure will be entirely new and not made of any used or reused components. They must comply with the Spanish and European Union legislation currently in force, and as the manufacturer's total responsibility, they must obtain the necessary certificates of homologation or declaration of conformity CE correspondent.\nThis procedure follows Directive 2001/95/CE on the general safety of products (R.D. 1801/2003, of December 26th, on the general safety of products B.O.E. of January 10th, 2004).\nWhen the products have been manufactured in accordance with norm"

In [8]:
page_text

'1\nPLIEGO DE PRESCRIPCIONES TÉCNICAS\nEXPEDIENTE ANTICIPADO DE GASTO: AB-SER2-24-003\nSUMINISTRO DE ESTACIONES DE TRABAJO PARA DIAGNÓSTICO MEDIANTE IMAGEN DIGITAL DE ANATOMÍA\nPATOLÓGICA CON DESTINO A HOSPITALES DEL SERVICIO GALLEGO DE SALUD, SUSCEPTIBLE DE COFINANCIACIÓN\nPOR LA UNIÓN EUROPEA EN UN 60% A TRAVÉS DEL PROGRAMA DE GALICIA FEDER 2021-2027, OBJETIVO POLÍTICO\nOP 1, OBJETIVO ESPECÍFICO OE. 1.2 Y ACTUACIÓN 1.2.02\nLos productos presentados a este procedimiento serán totalmente nuevos y no podrán tener ningún componente\nusado o reutilizado. Deberán cumplir la normativa española y comunitaria vigente que sea de aplicación, siendo\nasimismo de total responsabilidad del proveedor la obtención de los certificados de homologación o declaración de\nconformidad CE correspondientes.\nSe ajustarán a la directiva 2001/95/CE relativa a la seguridad general de los productos (R.D. 1801/2003, de 26 de\ndiciembre, sobre seguridad general de los productos B.O.E. de 10 de enero de 2004).\nCu